In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np


In [2]:
class DCGAN():
    def __init__(self, rows, cols, channels, z = 10):
        # Input shape
        self.img_rows = rows
        self.img_cols = cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = z

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=256, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/dcgan_mnist_%d.png" % epoch)
        plt.close()


In [3]:
dcgan = DCGAN(28,28,1)
dcgan.train(epochs=5000, batch_size=256, save_interval=50)


Metal device set to: Apple M1 Pro
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 14, 14, 32)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 zero_padding2d (ZeroPadding  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 64)         256       
 ormalization)        

2022-11-13 15:00:16.458855: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-13 15:00:16.459688: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 6272)              68992     
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 batch_normalization_3 (Batc  (None, 14, 14, 128)      512       
 hNormalization)                                                 
                                                                 
 activation (Activation)     (None, 14, 14, 128)      

2022-11-13 15:00:16.999783: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-13 15:00:17.038893: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:00:17.606047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-13 15:00:19.160421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0 [D loss: 1.150067, acc.: 27.93%] [G loss: 0.667357]


...

4427 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.147833]
4428 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.168984]
4429 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.174630]
4430 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.180601]
4431 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.115335]
4432 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.110403]
4433 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.193964]
4434 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.228165]
4435 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.196666]
4436 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.212470]
4437 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.178327]
4438 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.167449]
4439 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.223436]
4440 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.166923]
4441 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.139802]
4442 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.175587]
4443 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.138945]
4444 [D loss: 

4569 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.124643]
4570 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.170260]
4571 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.073099]
4572 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.020741]
4573 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.946239]
4574 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.825076]
4575 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.919548]
4576 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.928773]
4577 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.962999]
4578 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.932739]
4579 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.947260]
4580 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.941066]
4581 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.934577]
4582 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.002808]
4583 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.012639]
4584 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.041768]
4585 [D loss: 0.000002, acc.: 100.00%] [G loss: 4.165999]
4586 [D loss: 

4711 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.038474]
4712 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.041319]
4713 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.036800]
4714 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.023330]
4715 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.060829]
4716 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.012917]
4717 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.958715]
4718 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.051407]
4719 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.984365]
4720 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.983528]
4721 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.129590]
4722 [D loss: 0.000000, acc.: 100.00%] [G loss: 4.031041]
4723 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.061641]
4724 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.004182]
4725 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.019287]
4726 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.078191]
4727 [D loss: 0.000001, acc.: 100.00%] [G loss: 4.127862]
4728 [D loss: 

4853 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.915235]
4854 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.886358]
4855 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.837972]
4856 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.846445]
4857 [D loss: 0.000000, acc.: 100.00%] [G loss: 3.820745]
4858 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.823206]
4859 [D loss: 0.000000, acc.: 100.00%] [G loss: 3.852041]
4860 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.886546]
4861 [D loss: 0.000000, acc.: 100.00%] [G loss: 3.787452]
4862 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.787802]
4863 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.769993]
4864 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.805491]
4865 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.830591]
4866 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.909052]
4867 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.907028]
4868 [D loss: 0.000000, acc.: 100.00%] [G loss: 3.815644]
4869 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.845879]
4870 [D loss: 

4995 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.907148]
4996 [D loss: 0.000000, acc.: 100.00%] [G loss: 3.822834]
4997 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.927310]
4998 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.907778]
4999 [D loss: 0.000001, acc.: 100.00%] [G loss: 3.915436]
